In [1]:
clean_up = True
%run stdPackages.ipynb
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
d['data'] = os.path.join(d['data'], 'IO2018')

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.


## Inventory investments

In [2]:
name = 'smallGR' # name of model
db_IO = GpyDB(f"{d['data']}\\IO_{name}") # load IO database named IO_name
with open(f"{d['data']}\\glob_{name}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)

Add time index to the IO data:

In [3]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db_IO.__setitem__(k, addT(db_IO.get(k), glob.db['t'].vals)) for k in db_IO.getTypes(['variable','scalar_variable'])];

Inventory modelling only requires specifying: (1) What sectors are inventory sectors, (2) the speed of adjustment for the inventory (modelled as AR1 proces without stochasticity).

In [4]:
itory = pd.Index(['itory'], name = 's')
aggregateDB.subset_db(db_IO, itory) # goes through all symbols in db_IO and only keep the elements that are in the inventory set
I = CGE_Production.Inventory(name = name, db_IO = db_IO, itory=itory, glob = glob) # initialize module from nesting tree and global settings
I.compile(); 
I.write();
m = I.run(exportTo=d['work'], ws =ws)

Export:

In [5]:
I.s.db = m.out_db

Export:

In [6]:
with open(os.path.join(d['data'], I.name), "wb") as file:
    pickle.dump(I,file)